## K-S tree logistic regression & random forest

In [2]:
# import packages

import pandas as pd
import numpy as np

# data preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

# models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

# performance metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

### 1. data preprocessing

#### 1.1 data overview

Read training and testing data.

In [3]:
data = pd.read_csv('../../data/original/train.csv')
train_features, test_features, train_label, test_label = train_test_split(data.iloc[:, 2 :], data.iloc[:, 1], test_size = 0.2, stratify = data.iloc[:, 1], random_state = 2024)

Features with `cat` as postfix in names are categorical features. Features with `bin` as postfix in names are binary features. Features without postfix in names are continuous features.

Features are divided into 4 groups: `ind`, `reg`, `car`, `calc`.

In [4]:
train_features.columns

Index(['ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat',
       'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat',
       'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11',
       'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01',
       'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
       'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11',
       'ps_calc_12', 'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin',
       'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin',
       'ps_calc_20_bin'],
      dtype='obj

#### 1.2 missing values

Values of `-1` mean missing values. Codes below deal with missing values.

In [5]:
# replace "-1" with "np.nan"
train_features = train_features.replace(-1, np.nan)
test_features = test_features.replace(-1, np.nan)

In [6]:
# ratio of missing values in train_features
train_features_missing_ratio = train_features.isnull().mean()
train_features_missing_ratio = train_features_missing_ratio[train_features_missing_ratio > 0] # only consider features with non-zero missing ratio
train_features_top_missing_ratio = train_features_missing_ratio.sort_values(ascending=False)
print("missing ratio in train_features:\n")
print(train_features_top_missing_ratio)

missing ratio in train_features:

ps_car_03_cat    0.690587
ps_car_05_cat    0.447329
ps_reg_03        0.180820
ps_car_14        0.071420
ps_car_07_cat    0.019373
ps_ind_05_cat    0.009868
ps_car_09_cat    0.000962
ps_ind_02_cat    0.000380
ps_car_01_cat    0.000181
ps_ind_04_cat    0.000141
ps_car_11        0.000006
ps_car_02_cat    0.000004
ps_car_12        0.000002
dtype: float64


In [7]:
# ratio of missing values in test_features
test_features_missing_ratio = test_features.isnull().mean()
test_features_missing_ratio = test_features_missing_ratio[test_features_missing_ratio > 0] # only consider features with non-zero missing ratio
test_features_top_missing_ratio = test_features_missing_ratio.sort_values(ascending=False)
print("missing ratio in train_features:\n")
print(test_features_top_missing_ratio)

missing ratio in train_features:

ps_car_03_cat    0.692145
ps_car_05_cat    0.449812
ps_reg_03        0.182043
ps_car_14        0.072344
ps_car_07_cat    0.019018
ps_ind_05_cat    0.009324
ps_car_09_cat    0.000932
ps_ind_02_cat    0.000294
ps_car_01_cat    0.000176
ps_ind_04_cat    0.000134
ps_car_02_cat    0.000025
ps_car_11        0.000017
dtype: float64


Features `ps_car_03_cat`, `ps_car_05_cat` and `ps_reg_03` have too high missing ratio in both training and testing set, directedly delete these two features.

In [8]:
columns_to_drop = ['ps_car_03_cat', 'ps_car_05_cat']

# record dropped columns for later K-S statistic calculation
dropped_ps_car_03_cat = train_features['ps_car_03_cat']
dropped_ps_car_05_cat = train_features['ps_car_05_cat']

train_features = train_features.drop(columns=columns_to_drop, errors='ignore') # ignore error if columns to delete don't exist
test_features = test_features.drop(columns=columns_to_drop, errors='ignore')

Other features have missing ratio lower than 0.08. Fill continuous features with medieans of these features. Fill categorical and binary features with modes of these features. 

In [9]:
# names of features with low missing radio
columns_to_fill_with_mode = ['ps_car_07_cat', 'ps_ind_05_cat', 'ps_car_09_cat',
                             'ps_ind_02_cat', 'ps_car_01_cat', 'ps_ind_04_cat',
                             'ps_car_02_cat']
columns_to_fill_with_median = ['ps_car_14', 'ps_car_11', 'ps_car_12', 'ps_reg_03']

# training set
train_features[columns_to_fill_with_mode] = train_features[columns_to_fill_with_mode].fillna(
    train_features[columns_to_fill_with_mode].mode().iloc[0]
)
train_features[columns_to_fill_with_median] = train_features[columns_to_fill_with_median].fillna(
    train_features[columns_to_fill_with_median].median()
)


# testing set
test_features[columns_to_fill_with_mode] = test_features[columns_to_fill_with_mode].fillna(
    test_features[columns_to_fill_with_mode].mode().iloc[0]
)
test_features[columns_to_fill_with_median] = test_features[columns_to_fill_with_median].fillna(
    test_features[columns_to_fill_with_median].median()
)

#### 1.3 K-S static

Calculating K-S static has no requirement of standardization and encoding. So dataframes `train_features`, obtained in section 1.2 in this notebook, can be used here.

Calculating K-S static requires label information. So dataframes `train_label`, obtained in section 1.1 in this notebook, can be used here.

In [10]:
ks_results = []

for feature_name in train_features.columns:
    # construct dataframe
    temp_data = {
        'feature': list(train_features[feature_name].values),
        'target': list(train_label.values)
    }
    temp_df = pd.DataFrame(temp_data)
    temp_df = temp_df.sort_values(by='feature')
    # calculate ks_statistic of the given feature
    if 'cat' not in str(feature_name) and 'bin' not in str(feature_name):
        temp_df['cum_positive'] = (temp_df['target'] == 1).cumsum() / (temp_df['target'] == 1).sum() # cdf of label 1
        temp_df['cum_negative'] = (temp_df['target'] == 0).cumsum() / (temp_df['target'] == 0).sum() # cdf of label 0
        temp_df['diff'] = np.abs(temp_df['cum_positive'] - temp_df['cum_negative'])
        ks_statistic = temp_df['diff'].max()
        max_diff_feature_value = temp_df.loc[temp_df['diff'].idxmax(), 'feature'] # value of the continuous feature at which maximum cdf difference is achieved
    else:
        category_binary_stats = temp_df.groupby('feature')['target'].value_counts(normalize=False).unstack(fill_value=0)
        label_counts = temp_df['target'].value_counts()
        category_binary_stats[0] = category_binary_stats[0] / label_counts[0]
        category_binary_stats[1] = category_binary_stats[1] / label_counts[1]
        category_binary_stats['abs_diff'] = (category_binary_stats[0] - category_binary_stats[1]).abs()
        ks_statistic = category_binary_stats['abs_diff'].max()
        max_diff_feature_value = category_binary_stats['abs_diff'].idxmax() # value of the categorical/binary feature at which maximum proportion difference is achieved

    ks_results.append({'feature_name': feature_name, 'ks_statistic': ks_statistic, 'max_diff_feature_value': max_diff_feature_value})

ks_df = pd.DataFrame(ks_results)
ks_df_sorted = ks_df.sort_values(by='ks_statistic', ascending=False)
print(ks_df_sorted)

      feature_name  ks_statistic  max_diff_feature_value
32       ps_car_13      0.110406                0.832079
20       ps_reg_03      0.092526                0.820823
5    ps_ind_06_bin      0.089346                0.000000
19       ps_reg_02      0.087222                0.500000
6    ps_ind_07_bin      0.077938                0.000000
31       ps_car_12      0.077305                0.374166
15   ps_ind_16_bin      0.072481                1.000000
23   ps_car_04_cat      0.069395                0.000000
34       ps_car_15      0.068128                3.162278
21   ps_car_01_cat      0.067624                7.000000
16   ps_ind_17_bin      0.067192                1.000000
22   ps_car_02_cat      0.063740                0.000000
18       ps_reg_01      0.058091                0.600000
14       ps_ind_15      0.055723                8.000000
4    ps_ind_05_cat      0.051454                0.000000
0        ps_ind_01      0.050644                2.000000
2        ps_ind_03      0.04535

Calculate K-S statistics of dropped features: `ps_car_03_cat` and `ps_car_05_cat`.

In [11]:
# dropped_ps_car_03_cat
temp_data = {
    'feature': list(dropped_ps_car_03_cat.values),
    'target': list(train_label.values)
}
temp_df = pd.DataFrame(temp_data)
temp_df = temp_df.sort_values(by='feature')
category_stats = temp_df.groupby('feature')['target'].value_counts(normalize=False).unstack(fill_value=0)
label_counts = temp_df['target'].value_counts()
category_stats[0] = category_stats[0] / label_counts[0]
category_stats[1] = category_stats[1] / label_counts[1]
category_stats['abs_diff'] = (category_stats[0] - category_stats[1]).abs()
ks_statistic = category_stats['abs_diff'].max()
print("K-S Statistic of dropped_ps_car_03_cat: ", ks_statistic)

# dropped_ps_car_05_cat
temp_data = {
    'feature': list(dropped_ps_car_05_cat.values),
    'target': list(train_label.values)
}
temp_df = pd.DataFrame(temp_data)
temp_df = temp_df.sort_values(by='feature')
category_stats = temp_df.groupby('feature')['target'].value_counts(normalize=False).unstack(fill_value=0)
label_counts = temp_df['target'].value_counts()
category_stats[0] = category_stats[0] / label_counts[0]
category_stats[1] = category_stats[1] / label_counts[1]
category_stats['abs_diff'] = (category_stats[0] - category_stats[1]).abs()
ks_statistic = category_stats['abs_diff'].max()
print("K-S Statistic of dropped_ps_car_05_cat: ", ks_statistic)

K-S Statistic of dropped_ps_car_03_cat:  0.06447520236299445
K-S Statistic of dropped_ps_car_05_cat:  0.03492610362113624


Calculated K-S statistic is stored in a dataframe named `ks_df_sorted`. Select features with K-S statistic higher than 0.01.

In [12]:
dropped_features = list(ks_df_sorted[ks_df_sorted['ks_statistic'] < 0.01]['feature_name'])

In [13]:
train_features = train_features.drop(columns=dropped_features, errors='ignore') # ignore error if columns to delete don't exist
test_features = test_features.drop(columns=dropped_features, errors='ignore')

The previously dropped feature `ps_car_03_cat` and `ps_car_05_cat` has K-S static greater than 0.01.

#### 1.4 standardization and encoding

Standardize continuous features.

In [14]:
# names of continuous features
continuous_features = [column_name for column_name in train_features.columns if '_cat' not in column_name and '_bin' not in column_name]

scaler = StandardScaler()
train_continuous = scaler.fit_transform(train_features[continuous_features])
test_continuous = scaler.transform(test_features[continuous_features])

Encode categorical features using one-hot encoding.

In [15]:
# names of categorical features
categorical_features = [column_name for column_name in train_features.columns if '_cat' in column_name]

encoder = OneHotEncoder(sparse=False)
train_categorical = encoder.fit_transform(train_features[categorical_features])
test_categorical = encoder.transform(test_features[categorical_features])

Binary features remain fixed.

In [16]:
# names of binary features
binary_features = [column_name for column_name in train_features.columns if '_bin' in column_name]

train_binary = train_features[binary_features].values
test_binary = test_features[binary_features].values

Integrate three kinds of features into final traning and testing set.

In [17]:
train_features_processed = np.hstack((train_continuous, train_categorical, train_binary))
test_features_processed = np.hstack((test_continuous, test_categorical, test_binary))

Variables `train_features_processed` and `test_features_processed` are numpy arrays.

In [18]:
type(train_features_processed), train_features_processed.shape

(numpy.ndarray, (476169, 185))

In [19]:
type(test_features_processed), test_features_processed.shape

(numpy.ndarray, (119043, 185))

### 2. data resampling

`train_features_processed` derived in section 1.4 in this notebook can be used here. Only `train_features_processed` should be resampled. Resampling `test_features_processed` will lead to wrong testing AUC.

In [20]:
train_features_processed = pd.DataFrame(train_features_processed)
type(train_features_processed), train_features_processed.shape

(pandas.core.frame.DataFrame, (476169, 185))

In [21]:
type(train_label), train_label.shape

(pandas.core.series.Series, (476169,))

`train_features_processed` and `train_label` are first divided into a 80% development set (dev_features, dev_label) and a 20% validation set (val_features, val_label).

In [22]:
dev_features, val_features, dev_label, val_label = train_test_split(train_features_processed, train_label, test_size = 0.2, stratify = train_label, random_state = 2024)

#### 2.1 under-sampling

Interatively undersampling majority label samples, training the logistic regression model/random forest model on the undersampled development set.

In [23]:
# initialize
best_auc = 0
best_ratio = None
performance_results = []
majority_class = 0
majority_class_count = dev_label.value_counts()[majority_class]

# undersampling ratio
step = 0.07
ratios = np.arange(1.0, 0.0, -step)

for ratio in ratios:
    # RandomUnderSampler
    rus = RandomUnderSampler(sampling_strategy={majority_class: int(ratio * majority_class_count)}, random_state=42)
    dev_features_resampled, dev_label_resampled = rus.fit_resample(dev_features, dev_label)

    # train logistic regression/random forest on the resampled data
    #model = LogisticRegression(max_iter=100, class_weight='balanced', C=1e-2, penalty='l2', solver='lbfgs')
    #model.fit(dev_features_resampled, dev_label_resampled)

    model = RandomForestClassifier(n_estimators=1200, min_samples_leaf=1000, max_leaf_nodes=100, n_jobs=-1)
    model.fit(dev_features_resampled, dev_label_resampled)

    # evaluate the trained model on the validation set
    val_label_pred_prob = model.predict_proba(val_features)[:, 1]
    auc = roc_auc_score(val_label, val_label_pred_prob)
    
    # record current performance and update best performance
    performance_results.append((ratio, auc))
    if auc > best_auc:
        best_auc = auc
        best_ratio = ratio

# show best performance and corresponded ratio
print(f"Best sampling ratio: {best_ratio:.2f}")
print(f"Best AUC: {best_auc:.4f}")

Best sampling ratio: 0.93
Best AUC: 0.6237


#### 2.2 over-sampling

Interatively oversampling minority label samples, training the logistic regression model/random forest model on the oversampled development set, until the model's performance deteriorates on the validation set.

Oversampling is based on the undersampled data.

In [24]:
majority_class = 0
majority_class_count = dev_label.value_counts()[majority_class]
rus = RandomUnderSampler(sampling_strategy={majority_class: int(0.93 * majority_class_count)}, random_state=42)
dev_features_undersampled, dev_label_undersampled = rus.fit_resample(dev_features, dev_label)

In [25]:
# initialize
best_auc = 0
best_ratio = None
performance_results = []
minority_class = 1
minority_class_count = dev_label.value_counts()[minority_class]

# oversampling ratio
step = 0.07
ratios = np.arange(1.0, 2.0, step)

for ratio in ratios:
    # RandomOverSampler
    rus = RandomOverSampler(sampling_strategy={minority_class: int(ratio * minority_class_count)}, random_state=42)
    dev_features_resampled, dev_label_resampled = rus.fit_resample(dev_features_undersampled, dev_label_undersampled) # oversampling bases on the undersampled data

    # train logistic regression on the resampled data
    #model = LogisticRegression(max_iter=100, class_weight='balanced', C=1e-2, penalty='l2', solver='lbfgs')
    #model.fit(dev_features_resampled, dev_label_resampled)

    model = RandomForestClassifier(n_estimators=1200, min_samples_leaf=1000, max_leaf_nodes=100, n_jobs=-1)
    model.fit(dev_features_resampled, dev_label_resampled)

    # evaluate the trained model on the validation set
    val_label_pred_prob = model.predict_proba(val_features)[:, 1]
    auc = roc_auc_score(val_label, val_label_pred_prob)
    
    # record current performance and update best performance
    performance_results.append((ratio, auc))
    if auc > best_auc:
        best_auc = auc
        best_ratio = ratio

# show best performance and corresponded ratio
print(f"Best sampling ratio: {best_ratio:.2f}")
print(f"Best AUC: {best_auc:.4f}")

Best sampling ratio: 1.00
Best AUC: 0.6239


### 3. model training and testing

#### 3.1 model training

Based on the undersampling ratio and oversampling ratio obtained in the last section, undersample and oversample training features. Then train a logistic regression model and a raondom forest model on the resampled train data.

In [26]:
# undersample majority class
majority_class = 0
majority_class_count = train_label.value_counts()[majority_class]
rus = RandomUnderSampler(sampling_strategy={majority_class: int(0.93 * majority_class_count)}, random_state=42)
train_features_processed_resampled, train_label_resampled = rus.fit_resample(train_features_processed, train_label)

# oversample minority class
minority_class = 1
minority_class_count = train_label_resampled.value_counts()[minority_class]
rus = RandomOverSampler(sampling_strategy={minority_class: int(1.0 * minority_class_count)}, random_state=42)
train_features_processed_resampled, train_label_resampled = rus.fit_resample(train_features_processed_resampled, train_label_resampled)

In [27]:
train_features_processed_resampled.shape

(444052, 185)

In [28]:
# train a logistic regression model on the resampled data
lr_params = {
    'C': [1e-3, 1e-2, 1e-1, 1, 1e1],
    'penalty': ['l2'],
    'solver': ['lbfgs', 'saga']
}

lr_model = LogisticRegression(max_iter=100, class_weight='balanced', C=1e-2, penalty='l2', solver='lbfgs')
lr_model.fit(train_features_processed_resampled, train_label_resampled)

LogisticRegression(C=0.01, class_weight='balanced')

In [29]:
# train a random forest model on the resampled data
rf_model = RandomForestClassifier(n_estimators=1200, min_samples_leaf=1000, max_leaf_nodes=100, n_jobs=-1)
rf_model.fit(train_features_processed_resampled, train_label_resampled)

RandomForestClassifier(max_leaf_nodes=100, min_samples_leaf=1000,
                       n_estimators=1200, n_jobs=-1)

#### 3.2 model testing

In [30]:
lr_model_pred_prob = lr_model.predict_proba(test_features_processed)[:, 1]
auc_score_lr = roc_auc_score(test_label, lr_model_pred_prob)
print(f"AUC of logistic regression: {auc_score_lr}")

rf_model_pred_prob = rf_model.predict_proba(test_features_processed)[:, 1]
auc_score_rf = roc_auc_score(test_label, rf_model_pred_prob)
print(f"AUC of random forest: {auc_score_rf}")

AUC of logistic regression: 0.6270988941593839
AUC of random forest: 0.6275041779330105
